In [1]:
#using Pkg
#Pkg.add(PackageSpec(name="JWAS",rev="master"))
#Pkg.develop(PackageSpec(path="/home/jovyan/rohan/Box Sync/JWAS.jl"))
#Pkg.free("JWAS")

In [2]:
using DataFrames              # package for working with data sets
using JWAS                    # package for Bayesian regression analyses, including BayesB and BayesCπ        
using JWAS:misc               # utility functions
using Distributions       
using Plots   # package for plotting 
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/JWAS/tbeXw.ji for JWAS [c9a035f4-d403-5e6b-8649-6be755bc4798]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


### Input marker and phenotype data

In [3]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end

function removeCols!(M,cols)
    return M[:, [!(i in cols) for i=1:size(M,2)]]
end

removeCols! (generic function with 1 method)

In [4]:
posQTL  = Int64.(vec(readdlm("posQTL.csv")))
beta    = readdlm("beta.csv")
M = readMatBin("genotypes.bin");

In [5]:
n,p = size(M)
simData  = readtable("phenotypes.csv",header=false,names=[:y])# reading in the simulated phenotypes into a data frame
phenData = DataFrame(id=1:n, y=simData[:y])
first(phenData,5)

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


,id,y
,Int64,Float64⍰
1,1,-3.56863
2,2,1.73437
3,3,2.31795
4,4,-0.264018
5,5,-3.13096


In [6]:
phenTrain = phenData[1001:end,:]
first(phenTrain,5)

,id,y
,Int64,Float64⍰
1,1001,-6.22029
2,1002,-0.952557
3,1003,-9.66847
4,1004,-0.959437
5,1005,1.48486


In [7]:
resVar = var(simData[:y])/2
genVar = resVar

12.835233241520479

### Run BayesC$\pi$ using JWAS

In [8]:
ids = string.(1:size(M,1))                     # ids in genotype file are sequential numbers 1...n
model  = build_model("y = intercept",resVar)   # give model (except for marker part)
add_genotypes(model,M,genVar,header=false,rowID=ids,G_is_marker_variance=true);

21834 markers on 2000 individuals were added.


In [9]:
?runMCMC

search: runMCMC ZeroMeanFullNormalCanon



```
runMCMC(model::MME,df::DataFrame;
        chain_length=1000,starting_value=false,burnin = 0,
        output_samples_frequency = 0,output_samples_file="MCMC_samples",
        printout_model_info=true,printout_frequency=100,
        methods="conventional (no markers)",Pi=0.0,estimatePi=false,
        single_step_analysis= false,pedigree = false,
        missing_phenotypes=false,constraint=false,
        update_priors_frequency::Int64=0,
        outputEBV=true,output_PEV=false,output_heritability=false)
```

**Run MCMC for Bayesian Linear Mixed Models with or without estimation of variance components.**

  * Available **methods** include "conventional (no markers)", "RR-BLUP", "BayesB", "BayesC", "Bayesian Lasso", and "GBLUP".
  * Single step analysis is allowed if **single*step*analysis** = `true` and **pedigree** is provided.
  * The **starting_value** can be provided as a vector of numbers for all location parameteres and marker effects, defaulting to `0.0`s.
  * The first **burnin** iterations are discarded at the beginning of a MCMC chain of length **chain_length**.
  * Save MCMC samples every **output*samples*frequency** iterations, defaulting to `false`, to files **output*samples*file**, defaulting to `MCMC_samples.txt`. MCMC samples for hyperparametes (variance componets) and marker effects are saved by default if **output*samples*frequency** is provided. MCMC samples for location parametes can be saved using `output_MCMC_samples()`. Note that saving MCMC samples too frequently slows down the computation.
  * In Bayesian variable selection methods, **Pi** for single-trait analyses is a number; **Pi** for multi-trait analyses is a dictionary such as `Pi=Dict([1.0; 1.0]=>0.7,[1.0; 0.0]=>0.1,[0.0; 1.0]=>0.1,[0.0; 0.0]=>0.1)`, defaulting to `all markers have effects (0.0)` in single-trait analysis and `all markers have effects on all traits` in multi-trait analysis. **Pi** is estimated if **estimatePi** = true
  * In multi-trait analysis, **missing_phenotypes**, defaulting to `true`, and **constraint** variance components, defaulting to `false`, are allowed. If **constraint**=true, constrain residual covariances between traits to be zeros.
  * Print out the model information in REPL if `printout_model_info=true`; print out the monte carlo mean in REPL with **printout_frequency**, defaulting to `false`.
  * Individual estimted breeding values (EBVs) are returned if **outputEBV**=`true`, defaulting to `true`. Heritability and genetic variances are returned if **output_heritability**=`true`, defaulting to `false`. Note that estimation of heritability is computaionally intensive.


In [10]:
MCMCFileNAME = "MCMCSamples"                  # place to put samples of marker effects
                                              # marker effect is set to zero if that locus is not in model
out=runMCMC(model, phenTrain,                  # tell JWAS to run analysis, for given model and data 
    Pi=0.99,                                  # intial value of π
    estimatePi=true,
    chain_length=600,                         # length of chain
    printout_frequency=5000,                  # how often to show progress of analysis 
    printout_model_info=true,                 # tell JWAS to show the options used in this analysis
    methods="BayesC",                         # tell JWAS to run a BayesC analysis
    output_samples_frequency=20,              # how often to output sampled quantities
    output_samples_file=MCMCFileNAME,         # file name to output sampled marker effects
    output_PEV=true
);

A Linear Mixed Model was build using model equations:

y = intercept

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

methods                                      BayesC
chain_length                                    600
burnin                                            0
estimatePi                                     true
estimateScale                                 false
starting_value                                false
printout_frequency                             5000
output_samples_frequency                         20
constraint                                    false
missing_phenotypes                             true
update_priors_frequency                           0

Hyper-parameters Information:

residual variances:                          12.835
genetic variances (genomic):                  0.000
marker effect variances:                     12.835
π                         

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:31


In [11]:
keys(out)

Base.KeySet for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of marker effects"
  "EBV_y"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [12]:
out["EBV_y"]

,ID,Estimate,PEV
,Any,Any,Float64
1,1,-0.202897,1.85227
2,2,2.91164,1.88301
3,3,-0.612686,3.16087
4,4,8.33118,2.78611
5,5,0.776793,2.27356
6,6,-1.11141,1.6728
7,7,4.18797,1.17498
8,8,0.29975,1.55328
9,9,0.433276,2.12371


In [13]:
res = GWAS("MCMCSamples_marker_effects_y.txt";header=true)

21834×2 Array{Any,2}:
 "1"      0.0
 "2"      0.0
 "3"      0.0
 "4"      0.0
 "5"      0.0
 "6"      0.0
 "7"      0.0
 "8"      0.0
 "9"      0.0
 "10"     0.0
 "11"     0.0
 "12"     0.0
 "13"     0.0
 ⋮           
 "21823"  0.0
 "21824"  0.0
 "21825"  0.0
 "21826"  0.0
 "21827"  0.0
 "21828"  0.0
 "21829"  0.0
 "21830"  0.0
 "21831"  0.0
 "21832"  0.0
 "21833"  0.0
 "21834"  0.0

In [14]:
[res[posQTL,:] beta  out["Posterior mean of marker effects"][posQTL,2]]

40×4 Array{Any,2}:
 "8729"   0.9         1.13899     1.022      
 "18201"  0.4         0.742589    0.331738   
 "16771"  0.0        -0.416029    0.0        
 "14237"  0.0        -0.68567    -0.00126742 
 "11291"  0.0        -1.25694    -0.00415137 
 "15837"  0.0         0.060424    0.0        
 "17008"  0.0        -0.333503    0.0        
 "16115"  0.0         0.457435    0.0        
 "1681"   0.0         0.0351858   0.0        
 "15679"  0.0         2.7329      0.00850048 
 "2790"   0.0333333  -0.692791   -0.00183932 
 "20843"  0.0        -0.411793   -0.000687754
 "19915"  0.0        -0.146152   -0.000698686
 ⋮                                           
 "11391"  0.0        -0.132497   -0.000312758
 "17889"  0.0         0.426983    0.00355595 
 "7815"   0.466667   -1.64235    -0.673411   
 "18247"  0.0        -1.10174    -0.0110182  
 "21776"  0.0        -1.16958    -0.00286356 
 "1206"   0.0         0.15797     0.0        
 "6447"   0.366667   -0.978558   -0.380544   
 "14881"  0.0  

In [15]:
winVar = GWAS("MCMCSamples_marker_effects_y.txt",model.output_genotypes;header=true,window_size=100,threshold=0.001)

Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


,win,wStart,wEnd,wSize,prGenVar,WPPA
,Int64,Int64,Int64,Int64,Float64,Float64
1,1,1,100,100,0.01,0.0333333
2,2,101,200,100,0.07,0.0666667
3,3,201,300,100,0.71,0.366667
4,4,301,400,100,0.02,0.0666667
5,5,401,500,100,0.05,0.1
6,6,501,600,100,0.28,0.166667
7,7,601,700,100,0.12,0.166667
8,8,701,800,100,0.0,0.0
9,9,801,900,100,0.07,0.0333333


In [16]:
sortPosQTL = sort(posQTL);

In [17]:
PPA = 0.3
bigPPA = winVar[PPA .<= winVar[:WPPA],: ]
lowPos  = [findlast(sortPosQTL .<= row[2]) for row in eachrow(bigPPA)] 
highPos = [findfirst(sortPosQTL .>= row[3]) for row in eachrow(bigPPA)]   
wPos = [findfirst(bigPPA[i,2] .<= sortPosQTL .< bigPPA[i,3]) for i=1:size(bigPPA,1) ]

lowQTL  = [i == nothing ? 0 : sortPosQTL[i] for i in lowPos]
highQTL = [i == nothing ? 0 : sortPosQTL[i] for i in highPos]
wQTL    = [i == nothing ? 0 : sortPosQTL[i] for i in wPos]

res = DataFrame(
    wStart = bigPPA[:wStart],
    wEnd = bigPPA[:wEnd],
    wQTL = wQTL,
    oQTL = min.(bigPPA[:wStart]-lowQTL,highQTL-bigPPA[:wEnd]),
    prVar  = bigPPA[:prGenVar],
    WPPA   = bigPPA[:WPPA]
    )

,wStart,wEnd,wQTL,oQTL,prVar,WPPA
,Int64,Int64,Int64,Int64,Float64,Float64
1,201,300,285,201,0.71,0.366667
2,1101,1200,0,6,0.95,0.466667
3,2701,2800,2790,339,0.78,0.3
4,3101,3200,3139,311,9.1,1.0
5,3501,3600,0,243,0.85,0.533333
6,3801,3900,3843,355,4.6,1.0
7,4201,4300,4255,49,2.85,0.766667
8,6401,6500,6447,1205,4.61,1.0
9,7801,7900,7815,96,5.87,1.0


In [19]:
res[sortperm(res[:WPPA]),:]


,wStart,wEnd,wQTL,oQTL,prVar,WPPA
,Int64,Int64,Int64,Int64,Float64,Float64
1,2701,2800,2790,339,0.78,0.3
2,12601,12700,12691,295,0.48,0.3
3,17801,17900,17889,301,0.51,0.3
4,12701,12800,0,10,0.55,0.333333
5,201,300,285,201,0.71,0.366667
6,11901,12000,0,510,0.65,0.366667
7,1101,1200,0,6,0.95,0.466667
8,9301,9400,0,273,1.24,0.5
9,3501,3600,0,243,0.85,0.533333
